In [1]:
import pandas as pd
import plotly.express as px
from datasets import load_dataset
from plotly.subplots import make_subplots
import os
from transformers import AutoTokenizer
from tqdm import tqdm
import plotly.graph_objects as go


/opt/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("coastalcph/tydi_xor_rc")
ds_val = ds["validation"].to_pandas()
ds_train = ds["train"].to_pandas()

In [3]:
print(len(ds_train))
print(len(ds_val))
#tokenize 
#we load the llama3 tokenizer

15326
3028


In [4]:
#tokenize 
#we load the llama3 tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")

In [5]:
text_cols = ['question', 'context', 'answer']

for col in tqdm(text_cols):
    ds_train[f"{col}_n_tokens"] = ds_train[col].apply(lambda x: len(tokenizer.encode(x)))
    ds_val[f"{col}_n_tokens"] = ds_val[col].apply(lambda x: len(tokenizer.encode(x)))

100%|██████████| 3/3 [00:05<00:00,  1.86s/it]


In [6]:
#Summarize basic data statistics for train-
#ing and validation data in each of the languages Finnish (fi), Japanese
#(ja) and Russian (ru).

# distribution of answerable and not answerable questions
def create_language_plots(train_data, val_data, title_prefix):
    # Create subplots
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                        subplot_titles=(f"{title_prefix} - Training Data", 
                                        f"{title_prefix} - Validation Data"))

    # Only fi, ja, ru
    selected_langs = ['fi', 'ja', 'ru']
    dfs = [train_data, val_data]
    
    figs = []
    for df in dfs:
        selected_data = df[df['lang'].isin(selected_langs)]
        fig = px.sunburst(selected_data, path=['lang', 'answerable'],
                       color='lang',
                       color_discrete_map={True: 'rgba(0,100,0,0.7)', False: 'rgba(220,20,60,0.7)'})
        fig.update_traces(textinfo='label+percent parent')
        figs.append(fig)
    
    for fig in figs:
        fig.add_trace(fig.data[0], row=1, col=1)

    # Update layout
    fig.update_layout(title_text=f"{title_prefix} Language Distribution")

    return fig

import numpy as np

def create_language_histogram(train_data, val_data, title_prefix):
    selected_langs = ['fi', 'ja', 'ru']
    df = train_data[train_data['lang'].isin(selected_langs)]
    
    fig = make_subplots(rows=1, cols=3, subplot_titles=selected_langs)
    
    for i, lang in enumerate(selected_langs, 1):
        lang_data = df[df['lang'] == lang]
        
        # Answerable questions (True)
        answerable_data = lang_data[lang_data['answerable']]['question_n_tokens']
        # Non-answerable questions (False)
        non_answerable_data = lang_data[~lang_data['answerable']]['question_n_tokens']
        
        # Calculate bin edges for both distributions
        bins = np.histogram_bin_edges(np.concatenate([answerable_data, non_answerable_data]), bins=50)
        
        # Compute histograms
        answerable_hist, _ = np.histogram(answerable_data, bins=bins)
        non_answerable_hist, _ = np.histogram(non_answerable_data, bins=bins)
        
        # Normalize histograms
        answerable_hist = answerable_hist / answerable_hist.sum()
        non_answerable_hist = non_answerable_hist / non_answerable_hist.sum()
        
        # Plot normalized histograms
        fig.add_trace(
            go.Bar(x=bins[:-1], y=answerable_hist,
                   name=f'{lang} - Answerable',
                   marker_color='blue',
                   opacity=0.7),
            row=1, col=i
        )
        
        fig.add_trace(
            go.Bar(x=bins[:-1], y=non_answerable_hist,
                   name=f'{lang} - Non-answerable',
                   marker_color='red',
                   opacity=0.7),
            row=1, col=i
        )
        
        # Update x-axis and y-axis labels
        fig.update_xaxes(title_text="Number of Tokens", row=1, col=i)
        fig.update_yaxes(title_text="Relative Frequency" if i == 1 else None, row=1, col=i)
    
    fig.update_layout(
        title_text=f"{title_prefix} - Distribution of Question Token Counts by Language and Answerability",
        barmode='overlay',
        height=500,
        width=1200
    )
    
    return fig


os.makedirs("plots", exist_ok=True)
# Create and display the plot
fig_train = create_language_histogram(ds_train, ds_val, "Training Data")
fig_train.write_image("plots/week1_a_lang_token_distribution_normalized.png")

# Create and display plots for training data
""" fig_train = create_language_plots(ds_train, ds_val, "Training Data")
fig_train.write_image("plots/week1_a_dataset.png")
 """

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


' fig_train = create_language_plots(ds_train, ds_val, "Training Data")\nfig_train.write_image("plots/week1_a_dataset.png")\n '

In [11]:
#% (b) For each of the languages Finnish, Japanese and Russian, report the 5 most common 
#% words in the questions from the training set. What kind of words are they?



#collect all tokens from the questions, and count them


import MeCab

def get_top_words(df: pd.DataFrame, lang: str, n=5):
    df_lang = df[df['lang'] == lang].copy()
    
    if lang == 'ja':
        mecab = MeCab.Tagger("-Owakati")  # Initialize MeCab tokenizer
        df_lang.loc[:, 'words_question_tokens'] = df_lang['question'].apply(lambda x: mecab.parse(x).split())
    else:
        df_lang.loc[:, 'words_question_tokens'] = df_lang['question'].apply(lambda x: x.split(' '))
    
    all_tokens = np.concatenate(df_lang['words_question_tokens'].values)
    unique, counts = np.unique(all_tokens, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    top_unique_tokens = unique[sorted_indices][:n]
    top_tokens_dict = {token: int(count) for token, count in zip(top_unique_tokens, counts[sorted_indices][:n])}
    return top_tokens_dict



def visualize_top_tokens(
    df : pd.DataFrame, 
    lang : str, 
    n=5
):
    top_tokens = get_top_words(df, lang, n)
    fig = px.bar(x=list(top_tokens.keys()), y=list(top_tokens.values()), labels={'x':'Tokens', 'y':'Counts'})
    fig.update_layout(title=f"Top {n} Tokens in {lang}")
    fig.show()
    return fig

for lang in ['fi', 'ja', 'ru']:
    fig = visualize_top_tokens(ds_train, lang, 5)
    fig.write_image(f"plots/week1_b_top_5_tokens_{lang}.png")
